In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df=pd.read_csv("heart_clean_final.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,62,0,0,138,294.0,1,1,106,0,1.9,1,3,2,0
1,58,0,0,100,248.0,0,0,122,0,1.0,1,0,2,1
2,58,1,0,114,318.0,0,2,140,0,3.5,0,3,1,0
3,54,1,0,122,286.0,0,0,116,1,3.2,1,2,2,0
4,71,0,0,112,149.0,0,1,125,0,1.6,1,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,60,0,2,120,178.0,1,1,100,0,0.0,2,0,2,1
179,64,1,2,140,335.0,0,1,158,0,0.0,2,0,2,0
180,68,0,2,120,211.0,0,0,115,0,1.5,1,0,2,1
181,44,0,2,108,141.0,0,1,175,0,0.6,1,0,2,1


In [5]:
indep=df.drop("target",axis=1)
dep=df["target"]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy','log_loss'],'max_features': ['sqrt','log2'],'n_estimators':[10,50,100,200,300,500],"max_depth":[5,10,20],"min_samples_split":[2,5,10],"min_samples_leaf":[1,2,4,10],"class_weight":["balanced","balanced_subsample"]} 
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1') 
grid.fit(X_train, y_train)

Fitting 5 folds for each of 2592 candidates, totalling 12960 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'class_weight': ['balanced', 'balanced_subsample'],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [5, 10, 20],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4, 10],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 50, 100, 200, 300, 500]},
             scoring='f1', verbose=3)

In [17]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [29]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}: 0.822886531661845


In [31]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[ 5  7]
 [ 3 46]]


In [33]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.62      0.42      0.50        12
           1       0.87      0.94      0.90        49

    accuracy                           0.84        61
   macro avg       0.75      0.68      0.70        61
weighted avg       0.82      0.84      0.82        61



In [35]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8367346938775511

In [37]:
table=pd.DataFrame.from_dict(re)

In [39]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.014917,0.001537,0.003205,0.000811,balanced,gini,5,sqrt,1,2,10,"{'class_weight': 'balanced', 'criterion': 'gin...",0.864865,0.823529,0.857143,0.823529,0.888889,0.851591,0.025191,2016
1,0.055154,0.009904,0.004710,0.000398,balanced,gini,5,sqrt,1,2,50,"{'class_weight': 'balanced', 'criterion': 'gin...",0.864865,0.888889,0.944444,0.857143,0.894737,0.890016,0.030657,117
2,0.105933,0.011553,0.006713,0.000510,balanced,gini,5,sqrt,1,2,100,"{'class_weight': 'balanced', 'criterion': 'gin...",0.894737,0.888889,0.914286,0.857143,0.894737,0.889958,0.018523,146
3,0.220225,0.017012,0.011613,0.000805,balanced,gini,5,sqrt,1,2,200,"{'class_weight': 'balanced', 'criterion': 'gin...",0.864865,0.888889,0.914286,0.857143,0.894737,0.883984,0.020707,377
4,0.356164,0.039863,0.016016,0.000314,balanced,gini,5,sqrt,1,2,300,"{'class_weight': 'balanced', 'criterion': 'gin...",0.864865,0.888889,0.918919,0.857143,0.894737,0.884910,0.022099,313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,0.066377,0.001400,0.004507,0.000316,balanced_subsample,log_loss,20,log2,10,10,50,"{'class_weight': 'balanced_subsample', 'criter...",0.888889,0.882353,0.812500,0.750000,0.894737,0.845696,0.056345,2137
2588,0.134951,0.003547,0.006504,0.000448,balanced_subsample,log_loss,20,log2,10,10,100,"{'class_weight': 'balanced_subsample', 'criter...",0.864865,0.848485,0.838710,0.727273,0.888889,0.833644,0.055835,2417
2589,0.305064,0.024404,0.014212,0.002639,balanced_subsample,log_loss,20,log2,10,10,200,"{'class_weight': 'balanced_subsample', 'criter...",0.888889,0.848485,0.774194,0.787879,0.864865,0.832862,0.044436,2435
2590,0.427532,0.028788,0.015929,0.000379,balanced_subsample,log_loss,20,log2,10,10,300,"{'class_weight': 'balanced_subsample', 'criter...",0.864865,0.848485,0.774194,0.787879,0.888889,0.832862,0.044436,2435
